In [9]:
import pandas as pd
import math
from collections import defaultdict

In [10]:
train_data = pd.read_csv('train.csv')
valid_data = pd.read_csv('valid.csv')
test_data = pd.read_csv('test.csv')

In [11]:
print(len(train_data), len(valid_data), len(test_data))

905894 113236 113237


In [12]:
train_data = pd.concat([train_data, valid_data], ignore_index=True)

In [13]:
# let's only use 1/10 of the train data
# train_data = train_data.sample(frac=0.1, random_state=42)

In [14]:
def cossim(s1, s2):
  return len(s1.intersection(s2))/(math.sqrt(len(s1))*math.sqrt(len(s2)))

In [15]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
itemSet = set([])
ratingDict = {}
meanRating = 0

for j in range(len(train_data)):
  u,i,r = train_data['user'][j], train_data['item'][j], train_data['rating'][j]
  if i not in itemSet:
        itemSet.add(i)
  itemsPerUser[u].add(i)
  usersPerItem[i].add(u)
  ratingDict[u,i] = r
  meanRating += r

meanRating/=len(train_data)
print(meanRating)

4.411299834172284


In [16]:
test_rating = {}
mse = 0

for j in range(len(test_data)):
    u,i,r = test_data['user'][j],test_data['item'][j],test_data['rating'][j]
    numerator = 0
    denominator = 0
    pred = meanRating

    if len(itemsPerUser[u]) != 0:
        for user in usersPerItem[i]:
            if user != u:
                sim = cossim(itemsPerUser[user],itemsPerUser[u])
                numerator += sim * ratingDict[user,i]
                denominator += sim
        
        pred = meanRating
        if denominator != 0 :
            pred = numerator/denominator

    test_rating[u,i] = (r,pred)
    mse+=(r - pred)**2

mse/=len(test_data)
print(mse) 

1.7234829469327653
